## Imports


In [54]:
import pandas as pd
import joblib
import altair as alt


In [2]:
data = joblib.load('/Users/jasminkazi/code/mizzle-toe/find-your-dream-job/fydjob/output/indeed_proc/processed_data.joblib')

### Exploration


In [3]:
data.shape

(7859, 14)

In [4]:
data.head()

,job_title,job_text,company,location,job_info,query_text,source,job_link,tag_language,reviews,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,(Junior) Data Engineer (f/m/x),Customlytics ist die führende App Marketing Be...,Customlytics GmbH,Berlin,(Junior) Data Engineer (f/m/x)\nCustomlytics G...,data science,scrape_json,NaN,en,NaN,"[junior, data, engineer, fmx, customlytics, gm...","[customlytics, ist, die, führende, app, market...","[Customlytics, ist, die, führende, App, Market...","[junior, data, engineer, fmx]"
1,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,Berlin,Aroundhome6 Bewertungen - Berlin,data science,scrape_json,NaN,en,NaN,"[aroundhome, bewertungen, berlin]","[responsibilities, as, working, student, mfx, ...","[Responsibilities, As, working, student, mfx, ...",[]
2,,Aufgaben\nAls Werkstudent (m/w/d) IT arbeitest...,Aroundhome,Berlin,"Aroundhome6 Bewertungen - Berlin\nTeilzeit, Pr...",data science,scrape_json,NaN,de,NaN,"[aroundhome, bewertungen, berlin, teilzeit, pr...","[aufgaben, als, werkstudent, mwd, it, arbeites...","[Aufgaben, Als, Werkstudent, mwd, IT, arbeites...",[]
3,,Startdatum\n\nAb sofort\n\nDeine Aufgaben\n\nD...,Alexander Thamm GmbH,Berlin,Alexander Thamm GmbH - Berlin\nFestanstellung,data science,scrape_json,NaN,de,NaN,"[alexander, thamm, gmbh, berlin, festanstellung]","[startdatum, ab, sofort, deine, aufgaben, dein...","[Startdatum, Ab, sofort, Deine, Aufgaben, Dein...",[]
4,Full Stack Developer (m/f/d),We’re Phiture: a leading mobile growth consult...,Phiture,BerlinKreuzberg,Full Stack Developer (m/f/d)\nPhiture - Berlin...,data science,scrape_json,NaN,en,NaN,"[full, stack, developer, mfd, phiture, berlink...","[were, phiture, a, leading, mobile, growth, co...","[Were, Phiture, a, leading, mobile, growth, co...","[full, stack, developer, mfd]"


In [5]:
data = data.drop(columns=['job_info_tokenized','job_text_tokenized_titlecase', 'job_title_tokenized'])

In [6]:
data.columns

Index(['job_title', 'job_text', 'company', 'location', 'job_info',
       'query_text', 'source', 'job_link', 'tag_language', 'reviews',
       'job_text_tokenized'],
      dtype='object')

### No. of...


In [7]:
data['job_title'].count()

7859

In [8]:
#trying out modifying the df so I don't have a df per chart
source_lang = data['tag_language'].value_counts()
type(source_lang)
source_lang= data.groupby('tag_language')
type(source_lang)

pandas.core.groupby.generic.DataFrameGroupBy

In [9]:
#Lanugauges
data['tag_language'].nunique()
source_lang = pd.DataFrame(data['tag_language'].value_counts()).reset_index()
source_lang = source_lang.rename(columns = {'index':'Posting Language', 'tag_language':'Count'})
type(source_lang)

pandas.core.frame.DataFrame

In [10]:
alt.Chart(source_lang).mark_bar().encode(
    x=alt.X('Count', stack ='normalize'),
    #y='Posting Language',
    color = 'Count'
)


alt.Chart(...)

In [11]:
alt.Chart(source_lang).transform_joinaggregate(
    Total='sum(Count)',
).transform_calculate(
    Percentage="datum.Count / datum.Total"
).mark_bar().encode(
    alt.X('Percentage:Q', axis=alt.Axis(format='.0%')),
    y='Posting Language:N'
)

alt.Chart(...)

In [12]:
#Companies
print('Hiring Companies: ', data['company'].nunique())
source_comp = pd.DataFrame(data['company'].value_counts().reset_index())
source_comp= source_comp.rename(columns = {'index':'Company', 'company':'Count'})
source_comp

Hiring Companies:  2647


,Company,Count
0,Amazon.com,324
1,Ball Aerospace,187
2,Microsoft,136
3,Google,90
4,NYU Langone Health,77
...,...,...
2642,SDSU Research Foundation,1
2643,Aptus Health,1
2644,Delta Air Lines,1
2645,Asimov,1


In [13]:
source_comp = source_comp.nlargest(30, 'Count')

bars = alt.Chart(source_comp).mark_bar().encode(
    x='Count:Q',
    y=alt.Y('Company:N', sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

(bars + text).properties(height=500)

alt.LayerChart(...)

In [14]:
#query_text
print('No of query_text: ',data['query_text'].nunique())
source_job = pd.DataFrame(data['query_text'].value_counts().reset_index())
source_job= source_job.rename(columns = {'index':'Job', 'query_text':'Count'})
source_job

No of query_text:  4


,Job,Count
0,data scientist,303
1,data science,295
2,business intelligence,290
3,data engineer,263


In [15]:
alt.Chart(source_job).mark_bar().encode(
    x='Count',
    y='Job'
)
#rule = alt.Chart(source_job).mark_rule(color='red').encode(
#    x='mean(Count):Q'
#)

alt.Chart(...)

In [16]:
type(data['job_text_tokenized'][0])

list

In [17]:
from fydjob import utils
utils.load_skills()     #loads the skills from JSON file


{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [18]:
# get all the skill sets
categories = utils.load_skills()
categories.keys()
bus_skills = set(utils.load_skills()['business'])
knowledge_skills = set(utils.load_skills()['knowledge'])
code_skills = set(utils.load_skills()['programming'])
soft_skills = set(utils.load_skills()['soft_skills'])


In [19]:
#get skill for the first token
skill_0 = []

for word in data['job_text_tokenized'][0]:
    if word in code_skills:
        skill_0.append(word)
        
skill_0

['sql',
 'python',
 'git',
 'redshift',
 'bigquery',
 'cloud',
 'aws',
 'sql',
 'python',
 'git',
 'redshift',
 'bigquery',
 'cloud',
 'aws',
 'aws',
 'sql']

In [20]:
# count the occurences
from collections import Counter
counter0 = Counter(data['job_text_tokenized'][0])

#occs = {x: counter[x] for x in counter.keys() & set(skill_0)}

bus_occs = {x: counter0[x] for x in counter0.keys() & set(bus_skills)}
code_occs = {x: counter0[x] for x in counter0.keys() & set(code_skills)}
knowledge_occs ={x: counter0[x] for x in counter0.keys() & set(knowledge_skills)}
soft_occs ={x: counter0[x] for x in counter0.keys() & set(soft_skills)}
    
print(bus_occs)
print(code_occs)
print(knowledge_occs)
print(soft_occs)


{'consulting': 2, 'accounting': 1, 'analytical': 2, 'business': 2, 'analyst': 2, 'industry': 4, 'analyse': 2, 'crm': 1, 'dashboards': 1, 'analytics': 5, 'analysis': 1, 'stakeholder': 1, 'reporting': 2, 'marketing': 10, 'management': 1, 'consult': 1}
{'cloud': 2, 'sql': 3, 'aws': 3, 'git': 2, 'bigquery': 2, 'redshift': 2, 'python': 2}
{'statistics': 1, 'science': 1, 'hardware': 2, 'optimization': 2}
{'responsible': 2, 'flexible': 3, 'time': 1, 'management': 1, 'vision': 2, 'motivation': 1}


In [21]:
jd_0 = set(data['job_text_tokenized'][5])
vacancy = data['job_text_tokenized'][5]
jd_0

{'a',
 'ability',
 'across',
 'adapt',
 'additional',
 'again',
 'all',
 'always',
 'an',
 'and',
 'apply',
 'are',
 'areas',
 'as',
 'associate',
 'associates',
 'at',
 'based',
 'be',
 'begin',
 'berlin',
 'best',
 'better',
 'bewerbung',
 'both',
 'care',
 'challenging',
 'client',
 'clinical',
 'clinics',
 'collection',
 'committed',
 'communication',
 'computer',
 'connect',
 'constant',
 'constantly',
 'contact',
 'countries',
 'data',
 'degree',
 'department',
 'desire',
 'detail',
 'development',
 'diesen',
 'disabilities',
 'do',
 'doctors',
 'documents',
 'dont',
 'driven',
 'due',
 'eeoaa',
 'employees',
 'employer',
 'enable',
 'english',
 'ensure',
 'environment',
 'essential',
 'europe',
 'european',
 'evaluating',
 'expansion',
 'external',
 'eye',
 'fast',
 'fluency',
 'focused',
 'following',
 'for',
 'function',
 'gerade',
 'germany',
 'geschaltet',
 'given',
 'global',
 'have',
 'here',
 'highly',
 'hospitals',
 'improving',
 'in',
 'include',
 'individuals',
 'initi

In [22]:
jd_0.intersection(bus_skills)

{'client', 'management'}

In [23]:
jd_0.intersection(code_skills)

set()

In [24]:
jd_0.intersection(soft_skills)

{'committed',
 'communication',
 'driven',
 'management',
 'proactive',
 'project',
 'training',
 'verbal'}

In [25]:
jd_0.intersection(knowledge_skills)

{'query', 'science'}

In [26]:
for skill in categories:
    print(skill, jd_0.intersection(categories[skill]))


business {'management', 'client'}
knowledge {'science', 'query'}
programming set()
soft_skills {'communication', 'proactive', 'training', 'project', 'committed', 'driven', 'verbal', 'management'}
tech_adjectives set()


In [27]:
intersects_know = jd_0.intersection(knowledge_skills)
intersects_code = jd_0.intersection(code_skills)
intersects_soft = jd_0.intersection(soft_skills)
intersect_bus = jd_0.intersection(bus_skills)

In [28]:
vacancy.count('xgboost')

0

In [29]:
occsurrences_know ={} 
for skill in intersects_know:
    occsurrences_know[skill] = vacancy.count(skill)
occsurrences_know


{'science': 1, 'query': 1}

In [30]:
occsurrences_code ={} 
for skill in intersects_code:
    occsurrences_code[skill] = vacancy.count(skill)
occsurrences_code

{}

In [31]:
occsurrences_soft ={} 
for skill in intersects_soft:
    occsurrences_soft[skill] = vacancy.count(skill)
occsurrences_soft

{'communication': 1,
 'proactive': 1,
 'training': 3,
 'committed': 2,
 'driven': 1,
 'project': 1,
 'verbal': 1,
 'management': 5}

In [32]:
occsurrences_bus ={} 
for skill in intersects_soft:
    occsurrences_bus[skill] = vacancy.count(skill)
occsurrences_bus


{'communication': 1,
 'proactive': 1,
 'training': 3,
 'committed': 2,
 'driven': 1,
 'project': 1,
 'verbal': 1,
 'management': 5}

In [33]:
categories

{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [34]:
for category in categories:
    print(category)

business
knowledge
programming
soft_skills
tech_adjectives


In [35]:
vacancy = data['job_text_tokenized'][5]
vacancy_set = set(vacancy)
matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
    
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  vacancy.count(s)
print(occ)


{'business': {'management': 5, 'client': 1}, 'knowledge': {'science': 1, 'query': 1}, 'programming': {}, 'soft_skills': {'communication': 1, 'proactive': 1, 'training': 3, 'project': 1, 'committed': 2, 'driven': 1, 'verbal': 1, 'management': 5}, 'tech_adjectives': {}}


In [36]:
data['job_text_tokenized']

0       [customlytics, ist, die, führende, app, market...
1       [responsibilities, as, working, student, mfx, ...
2       [aufgaben, als, werkstudent, mwd, it, arbeites...
3       [startdatum, ab, sofort, deine, aufgaben, dein...
4       [were, phiture, a, leading, mobile, growth, co...
                              ...                        
8881    [are, you, datadriven, we, at, netapp, believe...
8882    [pharmacyclics, is, committed, to, the, develo...
8883    [oath, a, subsidiary, of, verizon, is, a, valu...
8884    [we, are, the, bing, core, relevance, team, re...
8885    [comcasts, technology, amp, product, organizat...
Name: job_text_tokenized, Length: 7859, dtype: object

In [37]:
occ

{'business': {'management': 5, 'client': 1},
 'knowledge': {'science': 1, 'query': 1},
 'programming': {},
 'soft_skills': {'communication': 1,
  'proactive': 1,
  'training': 3,
  'project': 1,
  'committed': 2,
  'driven': 1,
  'verbal': 1,
  'management': 5},
 'tech_adjectives': {}}

In [38]:
for vacancy in data['job_text_tokenized'][:3]:
    set(vacancy)

In [39]:
type(data['job_text_tokenized'].values)

numpy.ndarray

In [40]:
categories

{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [41]:
#vacancy = data['job_text_tokenized'][5]


def get_skill_count(vacancy):
    # turn the vacancy into a set to compare the words with defined dictionary
    vacancy_set = set(vacancy)
    # get the matching skills of a job description to the skills defined within different categories of skillsets (tech, bus, softskill...)
    matching_skill_per_category = {}
    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    #count the occurences of every matching skill
    occurrences = {}
    for category in categories:
        occurrences[category] = {}
        for skill in matching_skill_per_category[str(category)]:
            occurrences[category][skill] =  vacancy.count(skill)
    return occurrences

In [42]:
requirement = {}
for index, vacancy in enumerate(data['job_text_tokenized']):
    #print(vacancy)
    requirement[index]= get_skill_count(vacancy)
    occurence =get_skill_count(vacancy)
    

In [43]:
requirement

{0: {'business': {'consulting': 2,
   'accounting': 1,
   'analytical': 2,
   'business': 2,
   'analyst': 2,
   'industry': 4,
   'analyse': 2,
   'crm': 1,
   'dashboards': 1,
   'analytics': 5,
   'analysis': 1,
   'stakeholder': 1,
   'reporting': 2,
   'marketing': 10,
   'management': 1,
   'consult': 1},
  'knowledge': {'statistics': 1,
   'science': 1,
   'hardware': 2,
   'optimization': 2},
  'programming': {'cloud': 2,
   'aws': 3,
   'sql': 3,
   'git': 2,
   'bigquery': 2,
   'redshift': 2,
   'python': 2},
  'soft_skills': {'responsible': 2,
   'flexible': 3,
   'time': 1,
   'management': 1,
   'vision': 2,
   'motivation': 1},
  'tech_adjectives': {'google': 6,
   'microsoft': 1,
   'quantitative': 1,
   'datadriven': 1}},
 1: {'business': {'analyst': 1,
   'marketing': 1,
   'dashboards': 1,
   'business': 3},
  'knowledge': {'statistics': 1,
   'api': 1,
   'etl': 1,
   'science': 2,
   'query': 1,
   'scrum': 1,
   'database': 1},
  'programming': {'tableau': 1, 'aws

In [44]:
#things to do for streamlit

# ranking of all skills for streamlit --> agregated view of each categ
# percentage of category per job offer (10% bus, 20% tech...)

In [45]:
#put all vacancies into one huge list
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job

In [46]:
len(all_vacancies)

4065741

In [47]:
vacancy_set = set(all_vacancies)
len(vacancy_set)

67123

In [48]:

matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  all_vacancies.count(s)
print(occ)

{'business': {'profitable': 84, 'visualization': 1189, 'commercial': 1168, 'analyst': 1444, 'solution': 870, 'analyze': 2034, 'strategy': 2170, 'bd': 19, 'reporting': 2032, 'entrepreneurship': 33, 'scm': 7, 'business': 13710, 'analytic': 916, 'industry': 3911, 'report': 979, 'entrepreneurial': 534, 'crm': 311, 'dashboards': 399, 'distribution': 305, 'analytics': 6358, 'sales': 2040, 'analysis': 9066, 'stakeholder': 226, 'management': 8711, 'consult': 147, 'consulting': 1159, 'advertising': 950, 'accounting': 388, 'kpi': 44, 'client': 2340, 'entrepreneur': 38, 'analytical': 3883, 'olap': 21, 'analyse': 167, 'strategical': 3, 'administrative': 497, 'managing': 1508, 'logistic': 120, 'measurable': 100, 'strategic': 2260, 'administration': 644, 'efficiency': 570, 'planning': 1859, 'visualisation': 40, 'hr': 341, 'feasibility': 177, 'marketing': 3491}, 'knowledge': {'pipelines': 984, 'nlp': 502, 'cluster': 159, 'cleansing': 73, 'hardware': 882, 'correlation': 49, 'svm': 61, 'cicd': 98, 'sec

In [49]:
def get_skill_aggr(all_vacancies):
    
    vacancy_set = set(all_vacancies)
    matching_skill_per_category = {}

    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    occ = {}
    for category in categories:
        occ[category] = {}
        for s in matching_skill_per_category[str(category)]:
            occ[category][s] =  all_vacancies.count(s)
    return occ

In [56]:
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job
total_occurences = get_skill_aggr(all_vacancies)
total_occurences

{'business': {'profitable': 84,
  'visualization': 1189,
  'commercial': 1168,
  'analyst': 1444,
  'solution': 870,
  'analyze': 2034,
  'strategy': 2170,
  'bd': 19,
  'reporting': 2032,
  'entrepreneurship': 33,
  'scm': 7,
  'business': 13710,
  'analytic': 916,
  'industry': 3911,
  'report': 979,
  'entrepreneurial': 534,
  'crm': 311,
  'dashboards': 399,
  'distribution': 305,
  'analytics': 6358,
  'sales': 2040,
  'analysis': 9066,
  'stakeholder': 226,
  'management': 8711,
  'consult': 147,
  'consulting': 1159,
  'advertising': 950,
  'accounting': 388,
  'kpi': 44,
  'client': 2340,
  'entrepreneur': 38,
  'analytical': 3883,
  'olap': 21,
  'analyse': 167,
  'strategical': 3,
  'administrative': 497,
  'managing': 1508,
  'logistic': 120,
  'measurable': 100,
  'strategic': 2260,
  'administration': 644,
  'efficiency': 570,
  'planning': 1859,
  'visualisation': 40,
  'hr': 341,
  'feasibility': 177,
  'marketing': 3491},
 'knowledge': {'pipelines': 984,
  'nlp': 502,
 

In [51]:
df = pd.DataFrame(total_occurences).reset_index()
df= df.rename(columns = {'index':'skill'})

df

,skill,business,knowledge,programming,soft_skills,tech_adjectives
0,profitable,84.0,NaN,NaN,NaN,NaN
1,visualization,1189.0,1189.0,NaN,NaN,NaN
2,commercial,1168.0,NaN,NaN,NaN,NaN
3,analyst,1444.0,NaN,NaN,NaN,NaN
4,solution,870.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
388,instagram,NaN,NaN,NaN,NaN,162.0
389,quantitative,NaN,NaN,NaN,NaN,2656.0
390,holistic,NaN,NaN,NaN,NaN,96.0
391,scientific,NaN,NaN,NaN,NaN,3633.0


In [57]:
if df['business'] >0:
    df['category'] = 'business'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [55]:
for category in df.columns[1:]:
    #print(df[category])
    if df.loc[df[category] > 0,:]:
              df['category']=category
#df.loc[df['a'] > 10, ['a','c']]

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [61]:
df_bus = df[['skill','business']].dropna().sort_values('business', ascending = False).rename(columns = {'business':'count'})
df_bus['category']='business'
df_bus

,skill,count,category
11,business,13710.0,business
21,analysis,9066.0,business
23,management,8711.0,business
19,analytics,6358.0,business
13,industry,3911.0,business
31,analytical,3883.0,business
46,marketing,3491.0,business
29,client,2340.0,business
39,strategic,2260.0,business
6,strategy,2170.0,business


In [62]:
df_know = df[['skill','knowledge']].dropna().sort_values('knowledge', ascending = False).rename(columns = {'knowledge':'count'})
df_know['category']='knowledge'
df_know

,skill,count,category
86,research,14657.0,knowledge
124,science,9889.0,knowledge
88,statistics,2849.0,knowledge
62,modeling,2794.0,knowledge
55,security,2527.0,knowledge
...,...,...,...
69,causality,8.0,knowledge
70,nn,5.0,knowledge
72,chi,4.0,knowledge
123,userfirst,4.0,knowledge


In [103]:
df_code = df[['skill','programming']].dropna().sort_values('programming', ascending = False).rename(columns = {'programming':'count'})
df_code['category']='programming'
df_code

,skill,count,category
230,python,3870.0,programming
136,sql,2440.0,programming
202,r,2188.0,programming
172,cloud,2081.0,programming
179,java,1479.0,programming
...,...,...,...
254,mysqlmariadb,2.0,programming
156,kubeflow,2.0,programming
169,rust,2.0,programming
140,birt,1.0,programming


In [107]:
df_soft = df[['skill','soft_skills']].dropna().sort_values('soft_skills', ascending = False).rename(columns = {'soft_skills':'count'})
df_soft['category']='soft skills'
df_soft

,skill,count,category
23,management,8711.0,soft skills
318,project,5199.0,soft skills
322,communication,5181.0,soft skills
315,time,3674.0,soft skills
271,responsible,3472.0,soft skills
...,...,...,...
313,trustful,6.0,soft skills
270,charismatic,4.0,soft skills
296,empathic,4.0,soft skills
34,strategical,3.0,soft skills


In [109]:
df_categ = df_bus.append(df_know).append(df_code).append(df_soft)


In [110]:
df_categ['category'].unique()

array(['business', 'knowledge', 'programming', 'soft skills'],
      dtype=object)

In [112]:
import numpy as np
source = df_categ
category = np.array(df_bus['category'].unique())


base= alt.Chart(source).mark_bar().encode(
    x=alt.X('count'),
    y=alt.Y('skill:N', sort='-x'))


# A dropdown filter
genre_dropdown = alt.binding_select(options=category)
#input_dropdown = alt.binding_select(options=['Europe','Japan','USA'])
genre_select = alt.selection_single(fields=['Skill'], bind=genre_dropdown, name="Genre")
#selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
#color = alt.condition(selection,
#                    alt.Color('Origin:N', legend=None),
#                    alt.value('lightgray'))

filter_genres = base.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")



In [118]:


category

array(['all', 'business', 'knowledge', 'programming', 'soft skills'],
      dtype=object)

In [123]:
source = df_categ.nlargest(100, 'count')
categs = np.array(df_categ['category'].unique())
category = np.insert(categs, 0, 'all')


#Dropdownbox
input_dropdown = alt.binding_select(options = category)
selection = alt.selection_single(fields = ['category'], bind = input_dropdown, name= 'Professional ')

alt.Chart(source).mark_bar().encode(
x = ('count:Q'),
y = alt.Y('skill',sort='-x'),
).add_selection(selection).transform_filter(
    selection)


alt.Chart(...)

In [79]:
from vega_datasets import data
movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={"Release_Date":"date"})
)
movies

UrlData({
  format: DataFormat({
    parse: {'Release_Date': 'date'}
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/movies.json'
})

In [80]:
movies_df = pd.read_json('https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/movies.json')

In [81]:
movies_df


,Title,US_Gross,Worldwide_Gross,US_DVD_Sales,Production_Budget,Release_Date,MPAA_Rating,Running_Time_min,Distributor,Source,Major_Genre,Creative_Type,Director,Rotten_Tomatoes_Rating,IMDB_Rating,IMDB_Votes
0,The Land Girls,146083.0,146083.0,NaN,8000000.0,Jun 12 1998,R,NaN,Gramercy,None,None,None,None,NaN,6.1,1071.0
1,"First Love, Last Rites",10876.0,10876.0,NaN,300000.0,Aug 07 1998,R,NaN,Strand,None,Drama,None,None,NaN,6.9,207.0
2,I Married a Strange Person,203134.0,203134.0,NaN,250000.0,Aug 28 1998,None,NaN,Lionsgate,None,Comedy,None,None,NaN,6.8,865.0
3,Let's Talk About Sex,373615.0,373615.0,NaN,300000.0,Sep 11 1998,None,NaN,Fine Line,None,Comedy,None,None,13.0,NaN,NaN
4,Slam,1009819.0,1087521.0,NaN,1000000.0,Oct 09 1998,R,NaN,Trimark,Original Screenplay,Drama,Contemporary Fiction,None,62.0,3.4,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,Zack and Miri Make a Porno,31452765.0,36851125.0,21240321.0,24000000.0,Oct 31 2008,R,101.0,Weinstein Co.,Original Screenplay,Comedy,Contemporary Fiction,Kevin Smith,65.0,7.0,55687.0
3197,Zodiac,33080084.0,83080084.0,20983030.0,85000000.0,Mar 02 2007,R,157.0,Paramount Pictures,Based on Book/Short Story,Thriller/Suspense,Dramatization,David Fincher,89.0,NaN,NaN
3198,Zoom,11989328.0,12506188.0,6679409.0,35000000.0,Aug 11 2006,PG,NaN,Sony Pictures,Based on Comic/Graphic Novel,Adventure,Super Hero,Peter Hewitt,3.0,3.4,7424.0
3199,The Legend of Zorro,45575336.0,141475336.0,NaN,80000000.0,Oct 28 2005,PG,129.0,Sony Pictures,Remake,Adventure,Historical Fiction,Martin Campbell,26.0,5.7,21161.0


In [82]:
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = ['Action', 'Adventure', 'Black Comedy', 'Comedy',
       'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
       'Romantic Comedy', 'Thriller/Suspense', 'Western']

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Rounded_IMDB_Rating = "floor(datum.IMDB_Rating)",
    Hundred_Million_Production =  "datum.Production_Budget > 100000000.0 ? 100 : 10",
    Release_Year = "year(datum.Release_Date)"
).transform_filter(
    alt.datum.IMDB_Rating > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA_Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide_Gross:Q', scale=alt.Scale(domain=(100000,10**9), clamp=True)),
    y='IMDB_Rating:Q',
    tooltip="Title:N"
)

genre_dropdown = alt.binding_select(options=genres)
genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

filter_genres = base.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")

filter_genres

alt.Chart(...)